In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from src.ingestion import download_from_drive

In [ ]:
import json # Open and read the JSON file 
with open("file.json", "r", encoding="utf-8") as f: 
    data = json.load(f) # Print the data 
    print(data)

In [ ]:
df_azionisti = download_from_drive("")

In [3]:
df_azionisti.head()

,Unnamed: 0,Ragione_sociale,BvD_ID_number,Azionista_-_Nome,Azionista_-_Cognome,N°_di_azionisti_presenti,Azionisti_Numero_BvD,GUO_Numero_BvD,GUO_%_Diretta,GUO_Codice_ISO_del_Paese,GUO_Città,DUO_Numero_BvD,DUO_%_Diretta,DUO_Codice_ISO_del_Paese,DUO_Città,CSH_Numero_BvD,CSH_%_Diretta,CSH_Codice_ISO_del_Paese,CSH_Città
0,1.0,RAINSHOW S.R.L.,IT03595550835,EMANUELA,BOLOGNA,3.0,ITPPBLGMNL89P55G273N,ITPPBLGMNL89P55G273N,"51,00",IT,PALERMO,ITPPBLGMNL89P55G273N,"51,00",IT,PALERMO,ITPPBLGMNL89P55G273N,"51,00",IT,PALERMO
1,NaN,NaN,NaN,MASSIMILIANO,ZINGALES,NaN,ITPPZNGMSM72S27F158K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,LAURETTA NUNZIATINA,LEPRINO,NaN,ITPPLPRLTT49S56D569I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,OTOFARMA S.P.A.,IT08431301210,GIOVANNA,INCARNATO,3.0,ITPPNCRGNN77B63F839Y,IT08431301210,"100,00",IT,NAPOLI,IT08431301210,"100,00",IT,NAPOLI,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,IT08424141219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
